Step 1: Load & initial audit

In [133]:
import pandas as pd
import numpy as np

# Step 1: Load & initial audit
file_path = "Airbnb_Open_Data.csv"

df = pd.read_csv(file_path)

df.head()

C:\Users\acer\AppData\Local\Temp\ipykernel_30824\243929437.py:7: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


,id,NAME,host id,host_identity_verified,host name,neighbourhood group,neighbourhood,lat,long,country,...,service fee,minimum nights,number of reviews,last review,reviews per month,review rate number,calculated host listings count,availability 365,house_rules,license
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,...,$193,10.0,9.0,10/19/2021,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...,NaN
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,...,$28,30.0,45.0,5/21/2022,0.38,4.0,2.0,228.0,Pet friendly but please confirm with me if the...,NaN
2,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,NaN,Elise,Manhattan,Harlem,40.80902,-73.94190,United States,...,$124,3.0,0.0,NaN,NaN,5.0,1.0,352.0,"I encourage you to use my kitchen, cooking and...",NaN
3,1002755,NaN,85098326012,unconfirmed,Garry,Brooklyn,Clinton Hill,40.68514,-73.95976,United States,...,$74,30.0,270.0,7/5/2019,4.64,4.0,1.0,322.0,NaN,NaN
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,...,$41,10.0,9.0,11/19/2018,0.10,3.0,1.0,289.0,"Please no smoking in the house, porch or on th...",NaN


Step 2: Duplicate Checking

In [122]:
# Step 2: Duplicate checking for listing id
id_dup_count = df["id"].duplicated().sum()
print(f"Duplicate id count: {id_dup_count}")

# Show sample duplicate ids if any
if id_dup_count > 0:
    dup_ids = df.loc[df["id"].duplicated(), "id"].head(10)
    print("Sample duplicate ids:", dup_ids.tolist())
else:
    print("No duplicate ids found.")


Duplicate id count: 541
Sample duplicate ids: [35506831, 35507383, 35507935, 35508488, 35509040, 35509592, 35510145, 35510697, 35511249, 35511802]


In [123]:
# Remove duplicate rows by listing id
before_rows = len(df)
df = df.drop_duplicates(subset=["id"]).reset_index(drop=True)
removed = before_rows - len(df)
print(f"Removed {removed} duplicate rows. Remaining rows: {len(df)}")


Removed 541 duplicate rows. Remaining rows: 102058


Step 3: Missing value checking

In [124]:
# Step 3: Missing value report
missing_summary = pd.DataFrame({
    "missing_count": df.isna().sum(),
})

missing_summary

,missing_count
id,0
NAME,250
host id,0
host_identity_verified,289
host name,404
neighbourhood group,29
neighbourhood,16
lat,8
long,8
country,532


Step 4: Remove Outliner

In [125]:
# Clean 'minimum nights': set negative or >90 to NaN
min_nights_col = "minimum nights"
if min_nights_col in df.columns:
    df[min_nights_col] = pd.to_numeric(df[min_nights_col], errors="coerce")
    invalid_mask = (df[min_nights_col] < 0) | (df[min_nights_col] > 90)
    df.loc[invalid_mask, min_nights_col] = np.nan
    print(f"Set {invalid_mask.sum()} invalid '{min_nights_col}' values to NaN")
else:
    print(f"Column '{min_nights_col}' not found")


Set 437 invalid 'minimum nights' values to NaN


In [126]:
# Clean 'availability 365': set negative or >365 to NaN
avail_col = "availability 365"
if avail_col in df.columns:
    df[avail_col] = pd.to_numeric(df[avail_col], errors="coerce")
    invalid_mask = (df[avail_col] < 0) | (df[avail_col] > 365)
    df.loc[invalid_mask, avail_col] = np.nan
    print(f"Set {invalid_mask.sum()} invalid '{avail_col}' values to NaN")
else:
    print(f"Column '{avail_col}' not found")


Set 3185 invalid 'availability 365' values to NaN


Step 5: Preparation for Filling Missing Value

In [127]:
# Reformat price and service fee to plain numeric
money_cols = ["price", "service fee"]
for col in money_cols:
    if col in df.columns:
        df[col] = (
            df[col]
            .astype(str)
            .str.replace("$", "", regex=False)
            .str.replace(",", "", regex=False)
        )
        df[col] = pd.to_numeric(df[col], errors="coerce")
        print(f"Reformatted '{col}' to numeric")
    else:
        print(f"Column '{col}' not found")


Reformatted 'price' to numeric
Reformatted 'service fee' to numeric


Step 6: Filling Missing Value

In [128]:
# Fill missing values
string_cols = [
    "host_identity_verified",
    "neighbourhood group",
    "country",
    "instant_bookable",
    "cancellation_policy",
]
float_max_cols = ["Construction year", "review rate number"]
float_mean_cols = [
    "price",
    "service fee",
    "number of reviews",
    "minimum nights",
    "reviews per month",
    "calculated host listings count",
    "availability 365",
]

for col in string_cols:
    if col in df.columns:
        fill_value = df[col].dropna().max()
        df[col] = df[col].fillna(fill_value)
        print(f"Filled missing '{col}' with max value: {fill_value}")
    else:
        print(f"Column '{col}' not found")

for col in float_max_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")
        fill_value = df[col].max()
        df[col] = df[col].fillna(fill_value)
        print(f"Filled missing '{col}' with max value: {fill_value}")
    else:
        print(f"Column '{col}' not found")

for col in float_mean_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")
        fill_value = df[col].mean()
        df[col] = df[col].fillna(fill_value)
        print(f"Filled missing '{col}' with mean value: {fill_value}")
    else:
        print(f"Column '{col}' not found")


Filled missing 'host_identity_verified' with max value: verified
Filled missing 'neighbourhood group' with max value: manhatan
Filled missing 'country' with max value: United States
Filled missing 'instant_bookable' with max value: True
Filled missing 'cancellation_policy' with max value: strict
Filled missing 'Construction year' with max value: 2022.0
Filled missing 'review rate number' with max value: 5.0
Filled missing 'price' with mean value: 625.355580438263
Filled missing 'service fee' with mean value: 125.0392493982414
Filled missing 'number of reviews' with mean value: 27.517948466257668
Filled missing 'minimum nights' with mean value: 7.112822438031634
Filled missing 'reviews per month' with mean value: 1.3754082792207791
Filled missing 'calculated host listings count' with mean value: 7.936936671286331
Filled missing 'availability 365' with mean value: 134.52848361696724


C:\Users\acer\AppData\Local\Temp\ipykernel_30824\1476786719.py:23: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(fill_value)


In [129]:
# Recheck missing values after cleaning
missing_summary_after = pd.DataFrame({
    "missing_count": df.isna().sum(),
})

missing_summary_after


,missing_count
id,0
NAME,250
host id,0
host_identity_verified,0
host name,404
neighbourhood group,0
neighbourhood,16
lat,8
long,8
country,0


Step 7: Remove Unused Column

In [130]:
# Drop unused columns and save cleaned dataframe
cols_to_drop = [
    "NAME",
    "host name",
    "neighbourhood",
    "country code",
    "last review",
    "house_rules",
    "license",
]

cleaned_data = df.drop(columns=cols_to_drop, errors="ignore")
print(f"cleaned_data shape: {cleaned_data.shape}")


cleaned_data shape: (102058, 19)


In [132]:
# List attributes with data type and null count
attr_summary = pd.DataFrame({
    "dtype": cleaned_data.dtypes.astype(str),
    "null_count": cleaned_data.isna().sum(),
}).sort_index()

attr_summary


,dtype,null_count
Construction year,float64,0
availability 365,float64,0
calculated host listings count,float64,0
cancellation_policy,object,0
country,object,0
host id,int64,0
host_identity_verified,object,0
id,int64,0
instant_bookable,bool,0
lat,float64,8
